# KAGGLE MACHINE LEARNING    
## How Many Bikes?

______________________________________________________________________________________________________________________

# I/ Import data

In [1]:
import numpy as np
%pylab inline
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from os import chdir
chdir ('/Users/flavievampouille/Documents/DS&BA/CENTRALE/Machine Learning/Kaggle/')

Populating the interactive namespace from numpy and matplotlib


### 1) Import train dataset

In [2]:
train = pd.read_csv('train.csv')

##### Data to predict
As the final error is calculated on log(cnt) we train the model on log(cnt) as well.

In [3]:
y = np.log ( train['cnt'] )
del train['cnt']

##### Remove data which are not usefull

In [4]:
del train['instant']
del train['dteday']

##### Visualize data

In [5]:
train.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0


### 2) Import test dataset

In [6]:
test = pd.read_csv('test.csv')
instantTest = test['instant']

##### Remove data which are not usefull

In [7]:
del test['instant']
del test['dteday']

##### Visualize data

In [8]:
test.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed
0,1,0,1,0,0,4,1,1,0.26,0.2273,0.56,0.3881
1,1,0,1,1,0,4,1,1,0.26,0.2727,0.56,0.0000
2,1,0,1,2,0,4,1,1,0.26,0.2727,0.56,0.0000
3,1,0,1,3,0,4,1,1,0.26,0.2576,0.56,0.1642
4,1,0,1,4,0,4,1,1,0.26,0.2576,0.56,0.1642


### 3) Cyclic data
##### Season, month, hour and weekday are obviously cyclic data

In [9]:
# season
train ['cos_season'] = cos ( 2 * pi * train['season'] / 4 )
train ['sin_season'] = sin ( 2 * pi * train['season'] / 4 )
del train ['season']

test ['cos_season'] = cos ( 2 * pi * test['season'] / 4 )
test ['sin_season'] = sin ( 2 * pi * test['season'] / 4 )
del test ['season']

# mnth
train ['cos_mnth'] = cos ( 2 * pi * train['mnth'] / 12 )
train ['sin_mnth'] = sin ( 2 * pi * train['mnth'] / 12 )
del train ['mnth']

test ['cos_mnth'] = cos ( 2 * pi * test['mnth'] / 12 )
test ['sin_mnth'] = sin ( 2 * pi * test['mnth'] / 12 )
del test ['mnth']

# hr
train ['cos_hr'] = cos ( 2 * pi * train['hr'] / 24 )
train ['sin_hr'] = sin ( 2 * pi * train['hr'] / 24 )
del train ['hr']

test ['cos_hr'] = cos ( 2 * pi * test['hr'] / 24 )
test ['sin_hr'] = sin ( 2 * pi * test['hr'] / 24 )
del test ['hr']

# weekday
train ['cos_weekday'] = cos ( 2 * pi * train['weekday'] / 7 )
train ['sin_weekday'] = sin ( 2 * pi * train['weekday'] / 7 )
del train ['weekday']

test ['cos_weekday'] = cos ( 2 * pi * test['weekday'] / 7 )
test ['sin_weekday'] = sin ( 2 * pi * test['weekday'] / 7 )
del test ['weekday']

##### Visualize new dataset

In [10]:
train.head()

,yr,holiday,workingday,weathersit,temp,atemp,hum,windspeed,cos_season,sin_season,cos_mnth,sin_mnth,cos_hr,sin_hr,cos_weekday,sin_weekday
0,0,0,0,1,0.24,0.2879,0.81,0.0,6.123234e-17,1.0,0.866025,0.5,1.000000,0.000000,0.62349,-0.781831
1,0,0,0,1,0.22,0.2727,0.80,0.0,6.123234e-17,1.0,0.866025,0.5,0.965926,0.258819,0.62349,-0.781831
2,0,0,0,1,0.22,0.2727,0.80,0.0,6.123234e-17,1.0,0.866025,0.5,0.866025,0.500000,0.62349,-0.781831
3,0,0,0,1,0.24,0.2879,0.75,0.0,6.123234e-17,1.0,0.866025,0.5,0.707107,0.707107,0.62349,-0.781831
4,0,0,0,1,0.24,0.2879,0.75,0.0,6.123234e-17,1.0,0.866025,0.5,0.500000,0.866025,0.62349,-0.781831


______________________________________________________________________________________________________________________

# II/ Tested Models

______________________________________________________________________________________________________________________

# 1) Linear models

### Ridge

In [11]:
from sklearn.linear_model import Ridge

In [14]:
estimator = Ridge()
param_grid = {"alpha" : [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95] }
bestEstimator = GridSearchCV(Ridge(), param_grid, n_jobs=-1, verbose=0, cv=20, scoring='mean_squared_error')

bestEstimator.fit(train,y)
scoreMse = cross_val_score(estimator, train, y, scoring='mean_squared_error').mean()
score = cross_val_score(estimator, train, y).mean()
print("ScoreMse = %.2f, Score = %.2f" % (scoreMse , score))

bestEstimator.best_params_

ScoreMse = -0.91, Score = 0.57


{'alpha': 0.95}

In [ ]:
logPredict = bestEstimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle log Ridge.csv",index=False)

##### Score 0.57, mse 0.91 --> no submission

______________________________________________________________________________________________________________________

### Lasso

In [16]:
from sklearn.linear_model import Lasso

In [17]:
estimator = Ridge()
param_grid = {"alpha" : [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95] }
bestEstimator = GridSearchCV(Ridge(), param_grid, n_jobs=-1, verbose=0, cv=20, scoring='mean_squared_error')

bestEstimator.fit(train,y)
scoreMse = cross_val_score(estimator, train, y, scoring='mean_squared_error').mean()
score = cross_val_score(estimator, train, y).mean()
print("ScoreMse = %.2f, Score = %.2f" % (scoreMse , score))

bestEstimator.best_params_

ScoreMse = -0.91, Score = 0.57


{'alpha': 0.95}

In [ ]:
logPredict = bestEstimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle log Lasso.csv",index=False)

##### Score 0.57, mse 0.91 --> no submission

______________________________________________________________________________________________________________________

### ElasticNet

In [19]:
from sklearn.linear_model import ElasticNet

In [22]:
estimator = ElasticNet()
param_grid = {"alpha" : [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95] ,
              "l1_ratio" : [.1, .5, .7, .9, .95, .99, 1] , "normalize" : [True,False] }
bestEstimator = GridSearchCV(ElasticNet(), param_grid, n_jobs=-1, verbose=0, cv=20, scoring='mean_squared_error')
bestEstimator.fit(train,y)

score = cross_val_score(estimator, train, y, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

bestEstimator.best_params_

Score = -2.28


{'alpha': 0.05, 'l1_ratio': 0.1, 'normalize': False}

In [16]:
logPredict = bestEstimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle log ElasticNet.csv",index=False)

##### Score = 0.53, mse 2.28  --> no submission

______________________________________________________________________________________________________________________

### BayesianRidge

In [22]:
from sklearn.linear_model import BayesianRidge

In [32]:
estimator = ElasticNet()
param_grid = { "normalize" : [True,False] }
bestEstimator = GridSearchCV(ElasticNet(), param_grid, n_jobs=-1, verbose=0, cv=20, scoring='mean_squared_error')
bestEstimator.fit(train,y)

score = cross_val_score(bestEstimator, train, y, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

bestEstimator.best_params_

Score = -2.28


{'normalize': False}

In [ ]:
logPredict = bestEstimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle log BayesianRidge.csv",index=False)

##### Score : mse 2.28 --> no submission

______________________________________________________________________________________________________________________

### LogisticRegression ?? classifier

In [25]:
from sklearn.linear_model import LogisticRegression

______________________________________________________________________________________________________________________

### StochasticGradientDescent

In [23]:
from sklearn.linear_model import SGDRegressor

In [24]:
estimator = SGDRegressor()
param_grid = {"loss" : ["squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"] , 
              "penalty" : ["none", "l2", "l1", "elasticnet"] ,
              "l1_ratio" : [.1, .15, .2, .3, .4, .5, .7, .9, .95, .99, 1] }
bestEstimator = GridSearchCV(SGDRegressor(), param_grid, n_jobs=-1, verbose=0, scoring='mean_squared_error')

bestEstimator.fit(train,y)
score = cross_val_score(estimator, train, y, cv=20, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

bestEstimator.best_params_

Score = -0.92


{'l1_ratio': 0.15,
 'loss': 'squared_epsilon_insensitive',
 'penalty': 'elasticnet'}

In [ ]:
logPredict = bestEstimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log SGDRegressor.csv",index=False)

##### Score 0.54, mse 0.92 -> no submission

______________________________________________________________________________________________________________________

# 2) Kernel Ridge Regression
### KernelRidge

In [ ]:
from sklearn.kernel import KernelRidge

In [ ]:
estimator = KernelRidge().fit(train,y)
score = cross_val_score(estimator, train, y, cv=20, scoring='mean_squared_error', verbose=0, n_jobs=-1).mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log SGDRegressor.csv",index=False)

##### score 

______________________________________________________________________________________________________________________

# 3) K Nearest Neighbors
### KNeighborsRegressor

In [25]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
estimator = KNeighborsRegressor()
param_grid = {"n_neighbors" : range(3,1000,+2) , "algorithm" : ["auto", "ball_tree", "kd_tree"] }
bestEstimator = GridSearchCV(KNeighborsRegressor(), param_grid, scoring='mean_squared_error')

bestEstimator.fit(train,y)
score = cross_val_score(estimator, train, y, cv=100, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

bestEstimator.best_params_

In [ ]:
logPredict = bestEstimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log KNN 1000.csv",index=False)

##### Score 

______________________________________________________________________________________________________________________

# 4) Support Vector Machine
### SVR

In [12]:
from sklearn.svm import SVR

In [22]:
estimator = SVR(verbose=0).fit(train,y)
score = cross_val_score(estimator, train, y, cv=20, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

Score = -0.35


In [23]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log scoring=mse SVR.csv",index=False)

##### Score = 0.83, mse 0.35 --> Leaderboard = 0.57055

______________________________________________________________________________________________________________________

### NuSVR

In [ ]:
from sklearn.svm import NuSVR

In [18]:
estimator = NuSVR().fit(train,y)
score = cross_val_score(estimator, train, y, cv=20, verbose=0, n_jobs=-1, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

Score = 0.83


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  3.0min finished


In [19]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log NuSVR.csv",index=False)

##### Score = 0.83 --> Leaderboard  0.57331

______________________________________________________________________________________________________________________

# 5) Tree based methods

### DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
estimator = DecisionTreeRegressor().fit(train,y)
score = cross_val_score(estimator, train, y, cv=20, verbose=0, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log DecisionTreeRegressor.csv",index=False)

##### score 

______________________________________________________________________________________________________________________

### BaggingTreeRegressor

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
estimator = BaggingRegressor(n_estimators=1000, n_jobs=-1, verbose=0).fit(train,y)
score = cross_val_score(estimator, train, y, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log BaggingRegressor.csv",index=False)

##### score

______________________________________________________________________________________________________________________

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
estimator = RandomForestRegressor(n_estimators=1000, n_jobs=-1, verbose=0).fit(train,y)
score = cross_val_score(estimator, train, y, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log scoring=MSE RandomForestRegressor 1000.csv",index=False)

##### Score mse 0.06 --> leaderboard 0.40608 (BEST ONE)

______________________________________________________________________________________________________________________

### ExtraTreesRegressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
estimator = ExtraTreesRegressor(n_estimators=1000, n_jobs=-1, verbose=0).fit(train,y)
score = cross_val_score(estimator, train, y, cv=10, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log(cnt) cv=10 scoring=mse ExtraTreesRegressor.csv",index=False)

##### Score mse 0.03 --> Leaderboard 0.40897

______________________________________________________________________________________________________________________

### AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
estimator = AdaBoostRegressor(n_estimators=1000, loss="square", n_jobs=-1, verbose=0).fit(train,y)
score = cross_val_score(estimator, train, y, cv=10, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cyclic log(cnt) cv=10 scoring=mse AdaBoostRegressor.csv",index=False)

##### score

______________________________________________________________________________________________________________________

### GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
estimator = GradientBoostingRegressor(n_estimators=1000).fit(train, y)
score = cross_val_score(estimator, train, y, scoring='mean_squared_error').mean()
print("Score = %.2f" % score)

In [ ]:
logPredict = estimator.predict(test)
predict = np.zeros(len(test))
for i in range (len(test)) :
    predict[i] = np.exp(logPredict[i])
predict = np.round(predict)  # !! some values are negatives
for i in range (len (instantTest)) :
    if ( predict[i] < 0 ) :
        predict[i] = 0

final = pd.DataFrame()
final["id"] = instantTest
final["Prediction"] = predict
final.to_csv("Kaggle Cycle GradientBoostingRegressor.csv",index=False)

##### score mse  (Score : 0.81 / Kaggle leaderboard : 0.84395 ---- OVERFITTING)

______________________________________________________________________________________________________________________

# 6) MLP Regressor